<a href="https://colab.research.google.com/github/priyanjanjb/Cat-or-Dog-Image-Classifier/blob/Classifier/CatOrDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow -q
!pip install numpy -q
!pip install matplotlib -q
!pip install pandas -q

!pip install opendatasets -q



In [7]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time #calculate time

import opendatasets as od

In [8]:
od.download('https://www.kaggle.com/datasets/dineshpiyasamara/cats-and-dogs-for-classification') #download dataset

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: priyanjanperera16
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/cats-and-dogs-for-classification


100%|██████████| 217M/217M [00:01<00:00, 133MB/s]


In [9]:
BATCH_SIZE = 32
IMAGE_SIZE = (128,128) #height and width

In [10]:
train_data_dir = "/content/cats-and-dogs-for-classification/cats_dogs/train"
test_data_dir = "/content/cats-and-dogs-for-classification/cats_dogs/test"

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                         batch_size =BATCH_SIZE,
                                                         inage_size = IMAGE_SIZE,
                                                         subset = "training",
                                                         validation_split = 0.1
                                                         seed = 42)